### ME2: AlexNet


#### Import necessary packages

This notebook demonstrates custom deep learning inference using NumPy for portability and clarity.

- **NumPy** is the core array library we use to implement convolution, pooling, activation, and linear layers from scratch.
- We still rely on PyTorch / torchvision for data loading and to access pretrained AlexNet weights.
- Supporting packages (einops for concise tensor algebra, tqdm for progress reporting) round out the tooling.

The imports below are grouped to highlight standard libraries, third-party utilities, and NumPy stride tricks used for patch extraction.


In [ ]:
# Standard library imports
from typing import Tuple

# Third-party imports
from einops import einsum
from torch.utils.data import DataLoader
from torchvision.datasets import ImageNet
from torchvision.models import AlexNet_Weights
from tqdm.auto import tqdm

# NumPy imports
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view

#### Load the weights and biases of AlexNet

In this section, we extract the pretrained weights and biases from torchvision's AlexNet model.

- The weights and biases are loaded using the default configuration from `AlexNet_Weights`.
- These parameters are stored in a dictionary and printed to verify the available keys.
- Our custom layers will use these pretrained values for inference, ensuring the model matches the original AlexNet architecture.

This step is essential for initializing all custom layers with correct parameters before running inference on the validation set.


In [17]:
weights_and_biases = AlexNet_Weights.DEFAULT.get_state_dict()
print(weights_and_biases.keys())

odict_keys(['features.0.weight', 'features.0.bias', 'features.3.weight', 'features.3.bias', 'features.6.weight', 'features.6.bias', 'features.8.weight', 'features.8.bias', 'features.10.weight', 'features.10.bias', 'classifier.1.weight', 'classifier.1.bias', 'classifier.4.weight', 'classifier.4.bias', 'classifier.6.weight', 'classifier.6.bias'])


#### Custom Modules and Mixins

We replace `torch.nn.Module`/`nn.Sequential` with lightweight NumPy-native
wrappers so the rest of the notebook can stay framework-free once the
weights are loaded. `NumpyModule` keeps the familiar "forward/
**call**/train/eval" surface, while `NumpySequential` chains layers just
like PyTorch's container.

The same section also defines mixins used by later layers:

- `WeightsAndBiasMixin` pulls pretrained tensors from the AlexNet state dict
  and converts them to NumPy.
- `PatchMixin` provides the shared sliding-window extraction that powers
  both convolution and pooling implementations.


In [18]:
class NumpyModule:
    """Minimal module interface for NumPy-based layers."""

    def __init__(self) -> None:
        super().__init__()
        self.training = True

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

    def forward(self, *args, **kwargs):
        raise NotImplementedError

    def train(self, mode: bool = True):
        self.training = mode
        return self

    def eval(self):
        return self.train(False)


class NumpySequential(NumpyModule):
    """Sequential container chaining NumpyModule layers."""

    def __init__(self, *layers):
        super().__init__()
        self.layers = list(layers)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def train(self, mode: bool = True):
        super().train(mode)
        for layer in self.layers:
            train_fn = getattr(layer, 'train', None)
            if callable(train_fn):
                train_fn(mode)
        return self


class PatchMixin:
    """Mixin for extracting patches from input arrays with a given kernel size and stride.
    Used for convolution and pooling operations on NumPy arrays."""

    def __init__(self, kernel_size: int, stride: int) -> None:
        """Initialize patch extraction parameters.
        Args:
            kernel_size (int): Size of the square kernel.
            stride (int): Stride for patch extraction.
        """
        super().__init__()
        self.kernel_size = kernel_size
        self.stride = stride

    def _patch_with_stride(self, x_pad: np.ndarray) -> np.ndarray:
        """Extract k x k patches from the input array with the given stride.
        Args:
            x_pad (np.ndarray): Input array of shape (b, c, h, w).
        Returns:
            np.ndarray: Array of shape (b, c, h/stride, w/stride, k, k) containing the extracted patches.
        """
        windows = sliding_window_view(  # type: ignore
            x_pad,
            window_shape=(self.kernel_size, self.kernel_size),
            axis=(-2, -1)  # type: ignore
        )
        return windows[:, :, ::self.stride, ::self.stride, :, :]


class WeightsAndBiasMixin:
    """Mixin for loading pretrained weights and biases from a state dict as NumPy arrays."""

    def __init__(self, *args, **kwargs) -> None:
        """Initialize the mixin (calls super)."""
        super().__init__(*args, **kwargs)

    def init_weights_and_bias(self, weight_loc: str, bias_loc: str) -> Tuple[np.ndarray, np.ndarray]:
        """Load weights and biases from the state dict.
        Args:
            weight_loc (str): Key for weights in the state dict.
            bias_loc (str): Key for biases in the state dict.
        Returns:
            Tuple[np.ndarray, np.ndarray]: NumPy arrays for weights and biases.
        """
        weight = weights_and_biases[weight_loc].detach().cpu().numpy()
        bias = weights_and_biases[bias_loc].detach().cpu().numpy()
        return weight, bias

#### Profiling Usage

All forward methods are wrapped with a lightweight timing decorator storing per-call events in `PROFILE_EVENTS`.

After running inference you can view an aggregate table:

```python
profile_summary()                 # default sort by total time
profile_summary(sort="mean")      # sort by mean per call
profile_summary(sort="max")       # highlight worst single call
profile_summary(top_n=5)          # show only top 5 layers
```

Columns:

- Total(ms): cumulative time across calls
- Mean(ms): average per invocation
- Max(ms): slowest single invocation
- Calls: number of times that layer's forward executed

To reset profiling data between experiments:

```python
PROFILE_EVENTS.clear()
```


In [19]:
# Profiling utilities: decorator to time forward methods
import time
from functools import wraps
from collections import defaultdict

PROFILE_EVENTS = []  # list of dicts: {"name": str, "elapsed": float}


def profile_forward(name: str | None = None):
    """Decorator to measure execution time of forward methods.

    Args:
        name: Optional override name. If None attempts to derive from self.__class__.__name__.
    """
    def outer(fn):
        @wraps(fn)
        def inner(self, *args, **kwargs):
            label = name or self.__class__.__name__
            start = time.perf_counter()
            out = fn(self, *args, **kwargs)
            end = time.perf_counter()
            PROFILE_EVENTS.append({"name": label, "elapsed": end - start})
            return out
        return inner
    return outer


def profile_summary(sort: str = "total", top_n: int | None = None):
    """Print aggregated profiling results.

    Args:
        sort: One of {"total", "mean", "calls", "max"} to control ordering.
        top_n: If provided, limit output rows.
    """
    if not PROFILE_EVENTS:
        print("No profiling data collected.")
        return
    agg = defaultdict(lambda: {"total": 0.0, "calls": 0, "max": 0.0})
    for ev in PROFILE_EVENTS:
        a = agg[ev["name"]]
        a["total"] += ev["elapsed"]
        a["calls"] += 1
        if ev["elapsed"] > a["max"]:
            a["max"] = ev["elapsed"]
    rows = []
    for name, stats in agg.items():
        mean = stats["total"] / stats["calls"]
        rows.append({"name": name, **stats, "mean": mean})
    rows.sort(key=lambda r: r[sort], reverse=True)
    if top_n is not None:
        rows = rows[:top_n]
    header = f"{'Layer':<22} {'Calls':>5} {'Total(ms)':>12} {'Mean(ms)':>12} {'Max(ms)':>12}"
    print(header)
    print('-' * len(header))
    for r in rows:
        print(f"{r['name']:<22} {r['calls']:>5} {r['total']*1000:>12.3f} {r['mean']*1000:>12.3f} {r['max']*1000:>12.3f}")
    total_time = sum(r['total'] for r in rows)
    print('-' * len(header))
    print(
        f"Total profiled time: {total_time*1000:.2f} ms across {len(PROFILE_EVENTS)} calls")

#### Load the data

In this section, we load the ImageNet validation dataset and prepare it for inference with our custom NumPy-based AlexNet implementation.

- We use torchvision's `ImageNet` class to access the validation split, applying the standard AlexNet preprocessing transforms.
- The DataLoader batches images; our custom `default_collate` converts tensors directly to NumPy arrays.
- Keeping everything in NumPy simplifies the educational focus (no device transfers or GPU specifics).

This setup enables end-to-end inference using only NumPy for tensor operations alongside pretrained weights.


In [20]:
def default_collate(batch):
    """
    Collate function converting incoming PyTorch tensors to NumPy arrays.

    We convert tensors early so all subsequent custom layers operate purely on
    NumPy arrays (no device transfers needed).
    """
    imgs, labels = zip(*batch)  # imgs: tuple[torch.Tensor], labels: tuple[int]
    imgs = [np.asarray(img.numpy(), dtype=np.float32) for img in imgs]
    imgs = np.stack(imgs, axis=0)                 # [B,3,224,224]
    labels = np.asarray(labels, dtype=np.int64)    # [B]
    return imgs, labels


# implement using ImageNet
imagenet_val = ImageNet(
    root="data/ImageNet1k",
    split="val",
    transform=AlexNet_Weights.IMAGENET1K_V1.transforms()
)

val_dataloader = DataLoader(
    imagenet_val,
    batch_size=256,
    num_workers=0,
    pin_memory=False,
    drop_last=False,
    collate_fn=default_collate
)

#### Define the custom Conv2d

We implement convolution with an explicit im2col + GEMM strategy:

1. (Optional) pad the input.
2. Use `sliding_window_view` to obtain a strided view of all k×k receptive fields.
3. Reshape (B, C, out_h, out_w, k, k) -> (B*out_h*out_w, C*k*k).
4. Reshape weights (C_out, C, k, k) -> (C_out, C*k*k).
5. Perform a single matrix multiply and reshape back.

Compared to a high-rank `einsum` over the patch tensor, this form usually maps better onto optimized BLAS routines, improving runtime for large kernels and early layers.


In [21]:
class CustomConv2d(WeightsAndBiasMixin, PatchMixin, NumpyModule):
    """
    Custom 2D Convolution layer using NumPy and an im2col + GEMM strategy.

    Steps:
      1. (Optional) Pad input.
      2. Extract sliding windows (im2col) with stride via `sliding_window_view`.
      3. Reshape patches to a 2D matrix (B*out_h*out_w, C*k*k).
      4. Reshape weights to (out_channels, C*k*k) and perform a matrix multiply.
      5. Reshape back to (B, out_channels, out_h, out_w) and add bias.

    This avoids a high-rank einsum over strided memory and leverages optimized BLAS.
    """

    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        kernel_size: int,
        stride: int = 1,
        padding: int = 0,
        weight_loc: str = '',
        bias_loc: str = '',
    ) -> None:
        """Initialize the convolution layer with parameters and pretrained weights/biases.
        Args:
            in_channels (int): Number of input channels.
            out_channels (int): Number of output channels.
            kernel_size (int): Size of the convolution kernel.
            stride (int, optional): Stride for convolution. Defaults to 1.
            padding (int, optional): Padding for input. Defaults to 0.
            weight_loc (str, optional): Key for weights in state dict.
            bias_loc (str, optional): Key for biases in state dict.
        """
        super().__init__(kernel_size, stride)
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding

        self.weight, self.bias = self.init_weights_and_bias(
            weight_loc, bias_loc)
        self.reset_parameters()

    def reset_parameters(self) -> None:
        """No-op for pretrained weights. Provided for API compatibility."""
        pass

    def _apply_padding(self, x: np.ndarray) -> np.ndarray:
        """Apply zero padding to the input array if required.
        Args:
            x (np.ndarray): Input array of shape (b, c, h, w).
        Returns:
            np.ndarray: Padded input array.
        """
        if self.padding == 0:
            return x
        return np.pad(
            x,
            pad_width=((0, 0), (0, 0), (self.padding, self.padding),
                       (self.padding, self.padding)),
            mode='constant',
            constant_values=0,
        )

    @profile_forward()
    def forward(self, x: np.ndarray) -> np.ndarray:
        """Perform the forward pass using im2col + matrix multiply.
        Args:
            x (np.ndarray): Input array of shape (B, C_in, H, W).
        Returns:
            np.ndarray: Output array of shape (B, C_out, H_out, W_out).
        """
        B, C, H, W = x.shape
        k = self.kernel_size
        x_pad = self._apply_padding(x)
        H_p, W_p = x_pad.shape[2:]
        out_h = (H_p - k) // self.stride + 1
        out_w = (W_p - k) // self.stride + 1

        windows = self._patch_with_stride(x_pad)

        # Reshape to (B*out_h*out_w, C*k*k)
        col = windows.reshape(B, C, out_h, out_w, k * k)
        col = col.transpose(0, 2, 3, 1, 4).reshape(
            B * out_h * out_w, C * k * k)

        # Weight reshape: (C_out, C*k*k)
        w_mat = self.weight.reshape(self.out_channels, C * k * k)

        # GEMM: (N, C*k*k) @ (C*k*k, C_out) -> (N, C_out)
        out_mat = col @ w_mat.T

        # Reshape back: (B, out_h, out_w, C_out) -> (B, C_out, out_h, out_w)
        out = out_mat.reshape(
            B, out_h, out_w, self.out_channels).transpose(0, 3, 1, 2)

        if self.bias is not None:
            out += self.bias[None, :, None, None]
        return out

#### Custom ReLU

A straightforward NumPy implementation of ReLU using `np.maximum`.


In [22]:
class CustomReLU(NumpyModule):
    """
    Custom ReLU activation layer using NumPy.

    Applies the Rectified Linear Unit (ReLU) function: f(x)=max(0,x) element-wise.
    """

    @profile_forward()
    def forward(self, x: np.ndarray) -> np.ndarray:
        """Apply ReLU to the input.
        Args:
            x (np.ndarray): Input tensor of any shape.
        Returns:
            np.ndarray: Same shape as input with negatives zeroed.
        """
        return np.maximum(x, 0.0)

#### Custom MaxPool2d

Implemented via the same patch extraction utility as convolution, followed by a reduction (`np.max`) over the spatial kernel dimensions.


In [23]:
class CustomMaxPool2d(PatchMixin, NumpyModule):
    """
    Custom Max Pooling layer using NumPy.

    Extracts spatial patches then takes the maximum over each patch.
    """

    def __init__(self, kernel_size: int, stride: int) -> None:
        super().__init__(kernel_size, stride)

    @profile_forward()
    def forward(self, x: np.ndarray) -> np.ndarray:
        """Apply max pooling.
        Args:
            x (np.ndarray): Input of shape (batch, channels, height, width).
        Returns:
            np.ndarray: Pooled output.
        """
        patched_windows = self._patch_with_stride(x)
        return np.max(patched_windows, axis=(-2, -1))

#### Custom Adaptive AvgPool2d

Reduces arbitrary spatial dimensions to a fixed target by averaging over partitioned regions computed with integer floor/ceil boundaries.


In [24]:
class CustomAdaptiveAvgPool2d(NumpyModule):
    """
    Adaptive Average Pooling layer implemented with NumPy.

    Reduces spatial dimensions to a target (H_out, W_out) by averaging over
    variable-sized input regions computed via integer partitioning.
    """

    def __init__(self, output_size: tuple[int, int]) -> None:
        super().__init__()
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            self.output_size = output_size

    @profile_forward()
    def forward(self, x: np.ndarray) -> np.ndarray:
        """Apply adaptive average pooling.
        Args:
            x (np.ndarray): Input of shape (batch, channels, height, width).
        Returns:
            np.ndarray: Output of shape (batch, channels, H_out, W_out).
        """
        b, c, h, w = x.shape
        out_h, out_w = self.output_size
        out = np.zeros((b, c, out_h, out_w), dtype=x.dtype)
        for i in range(out_h):
            h_start = int(np.floor(i * h / out_h))
            h_end = int(np.ceil((i + 1) * h / out_h))
            for j in range(out_w):
                w_start = int(np.floor(j * w / out_w))
                w_end = int(np.ceil((j + 1) * w / out_w))
                region = x[:, :, h_start:h_end, w_start:w_end]
                out[:, :, i, j] = region.mean(axis=(-2, -1))
        return out

#### Custom Linear Module

Uses `einsum` to express the matrix multiply `x W^T` cleanly, then adds the bias. Operates purely on NumPy arrays.


In [25]:
class EinopsLinear(WeightsAndBiasMixin, NumpyModule):
    """
    Custom Linear (fully connected) layer using NumPy and einops.

    Performs y = x W^T + b via einsum for clarity.
    """

    def __init__(self, in_features: int, out_features: int, weight_loc: str, bias_loc: str):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight, self.bias = self.init_weights_and_bias(
            weight_loc, bias_loc)

    @profile_forward()
    def forward(self, x: np.ndarray) -> np.ndarray:
        """Forward pass.
        Args:
            x (np.ndarray): Shape (batch, in_features).
        Returns:
            np.ndarray: Shape (batch, out_features).
        """
        y = einsum(x, self.weight, "b i, o i -> b o")
        if self.bias is not None:
            y = y + self.bias
        return y

#### AlexNet Class Implementation

Assembles the feature extractor, adaptive pooling, and classifier blocks using the custom NumPy-based layers defined above.


In [26]:
class AlexNet(NumpyModule):
    """
    Custom AlexNet implementation using NumPy for educational inference.

    Replicates the original AlexNet architecture with all major layers
    (convolution, pooling, linear, activation) implemented from scratch
    operating on NumPy arrays. Pretrained weights are loaded from the
    torchvision reference model.
    """

    def __init__(self, num_classes: int = 1000) -> None:
        super().__init__()
        self.features = NumpySequential(
            CustomConv2d(3, 64, kernel_size=11, stride=4, padding=2,
                         weight_loc='features.0.weight', bias_loc='features.0.bias'),
            CustomReLU(),
            CustomMaxPool2d(kernel_size=3, stride=2),
            CustomConv2d(64, 192, kernel_size=5, padding=2,
                         weight_loc='features.3.weight', bias_loc='features.3.bias'),
            CustomReLU(),
            CustomMaxPool2d(kernel_size=3, stride=2),
            CustomConv2d(192, 384, kernel_size=3, padding=1,
                         weight_loc='features.6.weight', bias_loc='features.6.bias'),
            CustomReLU(),
            CustomConv2d(384, 256, kernel_size=3, padding=1,
                         weight_loc='features.8.weight', bias_loc='features.8.bias'),
            CustomReLU(),
            CustomConv2d(256, 256, kernel_size=3, padding=1,
                         weight_loc='features.10.weight', bias_loc='features.10.bias'),
            CustomReLU(),
            CustomMaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = CustomAdaptiveAvgPool2d((6, 6))
        self.classifier = NumpySequential(
            EinopsLinear(256 * 6 * 6, 4096, weight_loc='classifier.1.weight',
                         bias_loc='classifier.1.bias'),
            CustomReLU(),
            EinopsLinear(4096, 4096, weight_loc='classifier.4.weight',
                         bias_loc='classifier.4.bias'),
            CustomReLU(),
            EinopsLinear(4096, num_classes, weight_loc='classifier.6.weight',
                         bias_loc='classifier.6.bias'),
        )

    @profile_forward("AlexNet.forward")
    def forward(self, x: np.ndarray) -> np.ndarray:
        """Forward pass.
        Args:
            x (np.ndarray): (batch, 3, 224, 224)
        Returns:
            np.ndarray: (batch, num_classes) class scores.
        """
        x = self.features(x)
        x = self.avgpool(x)
        b = x.shape[0]
        x = x.reshape(b, -1)  # flatten
        x = self.classifier(x)
        return x

#### Inference

Run a forward pass over the validation set, accumulating accuracy using NumPy operations.


In [27]:
model = AlexNet()
model.eval()

total = 0
correct = 0

# Estimate total batches for progress bar length
try:
    total_batches = len(val_dataloader)
except TypeError:
    total_batches = None

for images, labels in tqdm(val_dataloader, total=total_batches, desc="Evaluating", leave=True):
    # images, labels already NumPy from collate
    outputs = model.forward(images)  # (b, num_classes) np.ndarray
    predicted = np.argmax(outputs, axis=1)
    total += int(labels.shape[0])
    correct += int((predicted == labels).sum())
    running_acc = correct / total if total else 0.0
    tqdm.write(f"Running Acc: {running_acc:.4f}")

accuracy = correct / total if total else 0.0
print(f"Validation Accuracy: {accuracy:.4f}")

Evaluating:   1%|          | 1/196 [00:05<16:16,  5.01s/it]

Running Acc: 0.6836


Evaluating:   1%|          | 2/196 [00:09<16:01,  4.96s/it]

Running Acc: 0.7109


Evaluating:   2%|▏         | 3/196 [00:14<15:45,  4.90s/it]

...

Running Acc: 0.5653


Evaluating:  99%|█████████▉| 195/196 [15:34<00:04,  4.91s/it]

Running Acc: 0.5661


Evaluating: 100%|██████████| 196/196 [15:36<00:00,  4.78s/it]

Running Acc: 0.5656
Validation Accuracy: 0.5656


#### Profile Summary


In [ ]:
profile_summary()

Layer                  Calls    Total(ms)     Mean(ms)      Max(ms)
-------------------------------------------------------------------
AlexNet.forward          196   814450.369     4155.359     4448.856
EinopsLinear             588   401987.004      683.651     1777.427
CustomConv2d             980   384456.266      392.302      646.020
CustomMaxPool2d          588    14866.174       25.283       56.330
CustomReLU              1372    11949.694        8.710       34.999
CustomAdaptiveAvgPool2d   196      739.565        3.773        6.927
-------------------------------------------------------------------
Total profiled time: 1628449.07 ms across 3920 calls
